# SNOMED

SNOMED CT is a standarised clinical terminology consisting of >350,000 unique concepts. It is owned, maintained and distributed by SNOMED International.

Please visit and explore https://www.snomed.org/ to find out further information about the various SNOMED CT products and services which they offer.

-------

UK Edition files can be found via [NHS TRUD](https://isd.digital.nhs.uk/)

Download files via API coming soon...


--------

All raw files from SNOMED should be placed in the local directory [here](data/snomed)



# Part 1: Preprocessing SNOMED CT for MedCAT

Once you have downloaded a SNOMED release of interest. Store the zipped folder containing your respective SNOMED release in the current colab working directory.

The folder name should look like: `SnomedCT_InternationalRF2_PRODUCTION_20210131T120000Z.zip
`


### Import required packages

In [5]:
import zipfile
from medcat.utils.preprocess_snomed import Snomed

### Load the data
Please see the section: [Access to SNOMED CT release files](#access_to_snomed_ct) for how to retrieve the zipped SNOMED CT release.

In [6]:
# Assign a path to the zipped SNOMED CT release download. (skip this step if the folder is not zipped)
snomed_path = "uk_sct2cl_35.2.0_20221123000001Z.zip"  # Enter your zipped Snomed folder here
snomed_folder = snomed_path[:-4]  # The unzipped SNOMED CT folder path

In [23]:
with zipfile.ZipFile(snomed_path, 'r') as zip_ref:
    zip_ref.extractall(snomed_folder)

### Preprocess the release for MedCAT

In [7]:
# Initialise
snomed = Snomed(snomed_folder)

In [8]:
### Skip this step if your version of snomed is not the UK extension released >2021.
### Note: this step will only work with MedCAT v1.2.7+

snomed.uk_ext = True

#### Create a SNOMED DataFrame

We first preprocess SNOMED to fit the following format:


|cui|name|ontologies|name_status|description_type_ids|type_ids|
|--|--|--|:--:|:--:|--|
|101009|Quilonia ethiopica (organism)|SNOMED|P|organism|81102976|
.
.
.

`cui` - The concept unique identifier, this is simply the `SCTID`.

`name` - This include the name of the concept. The status of the name is given in `name_status`

`ontologies` - Always SNOMED. Alternatively you can change it to your specific edition.

`name_status` - The Fully specified name or FSN is denoted with a `P` - Primary Name. Each concept must be assigned only one Primary Name. These should be unique across all SCTID/cui to avoid confusion. A synonym or other description type is represented as a `A` - Alternative Name. This can be enriched with all possible names and abbreviations for a concept of interest.

`description_type_ids` - These are processed to be the Semantic Tags of the concept.

`type_ids` - This is simply a 10 digit Hash of the Semantic Tags




In [59]:
# Create SNOMED DataFrame 
df = snomed.to_concept_df()

In [60]:
df.head()

,cui,name,name_status,ontologies,description_type_ids,type_ids
0,181000000101,Patient National Health Service number unknown...,P,SNOMED-CT,finding,67667581
1,191000000104,Patient NHS no ? correct (finding),P,SNOMED-CT,finding,67667581
2,201000000102,FMed 136A-ask for service PH (finding),P,SNOMED-CT,finding,67667581
3,231000000108,Patient deregistered - medical record envelope...,P,SNOMED-CT,finding,67667581
4,241000000104,Medical record envelope kept for medicolegal r...,P,SNOMED-CT,finding,67667581


In [29]:
# inspect
df[df['cui'] == '101009']

,cui,name,name_status,ontologies,description_type_ids,type_ids
62430,101009,Quilonia ethiopica (organism),P,SNOMED-CT,organism,81102976
419247,101009,Quilonia ethiopica,A,SNOMED-CT,organism,81102976


In [30]:
# Optional - Create a SCTID to FSN dictionary
primary_names_only = df[df["name_status"] == 'P']
sctid2name = dict(zip(primary_names_only['cui'], primary_names_only['name']))
del primary_names_only

In [31]:
# Test with example SCTID
sctid2name['101009']

'Quilonia ethiopica (organism)'

#### SNOMED Relationships

In [32]:
all_snomed_relationships = snomed.list_all_relationships()

In [33]:
# List of the SCTID of all snomed relationships
all_snomed_relationships

['116680003',
 '260686004',
 '363704007',
 '363699004',
 '405814001',
 '363714003',
 '363698007',
 '116676008',
 '363713009',
 '246501002',
 '418775008',
 '408731000',
 '363589002',
 '408730004',
 '408732007',
 '246090004',
 '408729009',
 '363701004',
 '272741003',
 '405813007',
 '425391005',
 '424361007',
 '363700003',
 '260507000',
 '405815000',
 '424226004',
 '424244007',
 '363709002',
 '363703001',
 '246454002',
 '246513007',
 '370135005',
 '363702006',
 '116686009',
 '42752001',
 '263502005',
 '47429007',
 '118169006',
 '370133003',
 '118168003',
 '118171006',
 '410675002',
 '246075003',
 '424876005',
 '405816004',
 '246093002',
 '255234002',
 '371881003',
 '419066007',
 '704319004',
 '370131001',
 '719722006',
 '370130000',
 '704327008',
 '370132008',
 '370134009',
 '260870009',
 '704321009',
 '118170007',
 '704324001',
 '719715003',
 '718497002',
 '363710007',
 '704325000',
 '116680003',
 '363698007',
 '363703001',
 '260686004',
 '363714003',
 '246075003',
 '272741003',
 '246454

In [34]:
# Using the SCTID to name to inspect what the FSN (fully specified names) are:
for sctid in all_snomed_relationships:
    print(sctid2name[sctid])

Is a (attribute)
Method (attribute)
Procedure site (attribute)
Direct device (attribute)
Procedure site - Indirect (attribute)
Interprets (attribute)
Finding site (attribute)
Associated morphology (attribute)
Has interpretation (attribute)
Technique (attribute)
Finding method (attribute)
Temporal context (attribute)
Associated procedure (attribute)
Procedure context (attribute)
Subject relationship context (attribute)
Associated finding (attribute)
Finding context (attribute)
Direct substance (attribute)
Laterality (attribute)
Procedure site - Direct (attribute)
Using access device (attribute)
Using substance (attribute)
Direct morphology (attribute)
Access (attribute)
Procedure device (attribute)
Using device (attribute)
Using energy (attribute)
Indirect morphology (attribute)
Has intent (attribute)
Occurrence (attribute)
Revision status (attribute)
Pathological process (attribute)
Has focus (attribute)
Has specimen (attribute)
Due to (attribute)
Clinical course (attribute)
Associated

In [35]:
# save a specific relationship to json
# In the example we save the "IS a (attribute)" hierarchical relationship.
snomed.relationship2json("116680003", "ISA_relationship.json")

#### Classification maps to inbuilt external terminologies 

The UK maps provide a one directional link from SNOMED CT to OPCS-4 and ICD-10. The international edition will only link to ICD-10.

They are compiled to reflect the national clinical coding standards and aid the application of the three dimensions of coding accuracy:

- Individual codes
- Totality of codes; and
- Sequencing of codes

Four different types of map are provided to accommodate the different circumstances that may influence ICD-10/OPCS-4 code assignment. 


|Map Type 1|Map Type 2|Map Type 3|Map Type 4|
|---|---|---|---|
|Links a single SNOMED CT concept to a single classification code to represent the clinical meaning of the concept. |Links a single SNOMED CT concept to a combination of classification codes which collectively represents the meaning of the SNOMED CT concept. <br /><br /> Map Type 1 and 2 may be generated automatically within systems, allowing the coding expert to devote time to the validation of more complex maps.|Links a single SNOMED CT concept to a choice of classification codes (default and alternative targets). Validation involves a coding expert using the additional detail found within the medical record, applying the rules, conventions and standards of the classifications, and manually selecting the final classification code or codes from a list of alternative targets.|Links a single SNOMED CT concept to a choice of classifications maps. Each choice of map may contain a single, combination or choice of target codes. Final selection will be informed by additional detail within the medical record and application of classification expertise by the coder.|


##### Map Blocks, Map Groups and Map Priorities

Each classification map will contain at least one map block, one map group and one map priority. Map Blocks, Map Group and Map Priority are numbered sequentially, starting at 1.

- A __Map Block__ signifies a code or string of codes that represent the SNOMED CT concept’s
fully specified name (FSN). Multiple Map Blocks will be included within the map if it is
necessary to represent the concept in multiple ways (e.g. sequencing of dagger and asterisk
codes).
- A __Map Group__ signifies each individual target code within a Map Block. Each individual code
within a Map Block will be allocated to its own Map Group unless it is an Alternative code.
Where multiple codes are required, the Map Groups builds in any required classification
sequencing rules.
- A __Map Priority__ signifies the priority of the code within the group based on the order in which
the codes are presented within mapping tables to enable the information to be read by
computer software systems. In a complex map, where alternative targets are provided within
a block or a group, an ALTERNATIVE target code is always listed before the TRUE target
code.



Let's inspect and create a SNOMED to ICD-10 map to add to the MedCAT concept database (cdb) additional information section

##### ICD-10
For SNOMED to ICD-10 mapping read more on:
Map Blocks, Map Groups and Map Priorities, for correct official mapping methodology.

In [9]:
# ICD-10
icd_df = snomed.map_snomed2icd10()

In [10]:
icd_df.head()

,id,effectiveTime,active,moduleId,refsetId,referencedComponentId,mapGroup,mapPriority,mapRule,mapAdvice,mapTarget,correlationId,mapCategoryId
0,49b07dca-0559-59d9-8c85-3198c941a813,20130731,1,449080006,447562003,10000006,1,1,TRUE,ALWAYS R07.4,R07.4,447561005,447637006
1,2c141272-bbc0-5c34-aa48-72f2dc370ead,20150131,1,449080006,447562003,10001005,1,1,TRUE,ALWAYS A41.9,A41.9,447561005,447637006
2,47cf847c-c85c-5979-8129-98475d4ab274,20190731,1,449080006,447562003,10007009,1,1,TRUE,ALWAYS Q87.1 | POSSIBLE REQUIREMENT FOR ADDITI...,Q87.1,447561005,447637006
3,4cd08e27-0417-5c5d-8873-6a72d130e918,20140731,1,449080006,447562003,1001000119102,1,1,TRUE,ALWAYS I26.9 | MAPPED FOLLOWING WHO GUIDANCE,I26.9,447561005,447637006
4,ca792560-1589-5f24-9126-b040e8ea2b2d,20180731,1,449080006,447562003,10017004,1,1,TRUE,ALWAYS K03.0,K03.0,447561005,447637006


In [38]:
# drop codes with no mapping
icd_df = icd_df[icd_df['mapTarget']!='']

In [47]:
sctid2icd10 = icd_df.groupby('referencedComponentId').apply(lambda group: [{'code': row['mapTarget'],
                                                                                                    'mapGroup': row['mapPriority'],
                                                                                                    'mapPriority': row['mapPriority'],
                                                                                                    'mapRule': row['mapRule'],
                                                                                                    'mapAdvice': row['mapAdvice']} for _, row in group.iterrows()]).to_dict()

In [48]:
sctid2icd10

{'10000006': [{'code': 'R07.4',
   'mapGroup': '1',
   'mapPriority': '1',
   'mapRule': 'TRUE',
   'mapAdvice': 'ALWAYS R07.4'}],
 '10001005': [{'code': 'A41.9',
   'mapGroup': '1',
   'mapPriority': '1',
   'mapRule': 'TRUE',
   'mapAdvice': 'ALWAYS A41.9'}],
 '10007009': [{'code': 'Q87.1',
   'mapGroup': '1',
   'mapPriority': '1',
   'mapRule': 'TRUE',
   'mapAdvice': 'ALWAYS Q87.1 | POSSIBLE REQUIREMENT FOR ADDITIONAL CODE TO FULLY DESCRIBE DISEASE OR CONDITION'}],
 '1001000119102': [{'code': 'I26.9',
   'mapGroup': '1',
   'mapPriority': '1',
   'mapRule': 'TRUE',
   'mapAdvice': 'ALWAYS I26.9 | MAPPED FOLLOWING WHO GUIDANCE'}],
 '10017004': [{'code': 'K03.0',
   'mapGroup': '1',
   'mapPriority': '1',
   'mapRule': 'TRUE',
   'mapAdvice': 'ALWAYS K03.0'}],
 '100191000119105': [{'code': 'N42.8',
   'mapGroup': '1',
   'mapPriority': '1',
   'mapRule': 'TRUE',
   'mapAdvice': 'ALWAYS N42.8'}],
 '100211000119106': [{'code': 'R25.2',
   'mapGroup': '1',
   'mapPriority': '1',
   'ma

In [49]:
# To view the SNOMED to ICD-10 Map structure.
# The structure should be '100006': [ {'code': 'R07.4', name: 'diabetes type2', 'priority': 1}, etc]
sctid2icd10['84757009']

[{'code': 'G40.9',
  'mapGroup': '1',
  'mapPriority': '1',
  'mapRule': 'TRUE',
  'mapAdvice': 'ALWAYS G40.9'}]

##### OPCS
Office of Population Censuses and Surveys


__Note:__ only the SNOMED UK extension edition contains this information
Skip if your version is not a UK extension

In [50]:
opcs_df = snomed.map_snomed2opcs4()

In [51]:
opcs_df.head()

,id,effectiveTime,active,moduleId,refsetId,referencedComponentId,mapGroup,mapPriority,mapRule,mapAdvice,mapTarget,correlationId,mapBlock
0,3ab6f134-d3c2-56ee-ab00-6ea9d7d1f3e7,20200805,1,999000031000000106,999002271000000101,10000006,1,1,,ALWAYS R07.0,R070,447561005,1
1,0024b3af-dfde-5114-b8b1-b3a066e6a8f6,20200805,1,999000031000000106,999002271000000101,10000006,1,2,,ALWAYS R07.2,R072,447561005,1
2,7aed656d-4c6a-53b0-acac-5e6bd46ab8d4,20200805,1,999000031000000106,999002271000000101,10000006,1,3,,ALWAYS R07.1,R071,447561005,1
3,ac662d04-eb34-54c9-84a7-d1c2c0d6b21c,20200805,1,999000031000000106,999002271000000101,10000006,1,4,,ALWAYS R07.3,R073,447561005,1
4,d6a04ab8-fa16-5a5c-a5ea-cb4df1294395,20200805,1,999000031000000106,999002271000000101,10000006,1,5,,ALWAYS R07.4,R074,447561005,1


In [81]:
opcs_df['refsetId'].unique()  # notice how there are two codes?
# SCTID:'999002271000000101' represents ICD10 codes and SCTID:'1126441000000105' OPCS4
# Filtering by '999002271000000101' will also show more ICD10 codes. Explore why. Something funny with the UK ext


array(['999002271000000101', '1126441000000105'], dtype=object)

In [83]:
# Filter for just OPCS4
opcs_df = opcs_df[opcs_df['refsetId']=='1126441000000105']

In [84]:
sctid2opcs4 = opcs_df.groupby('referencedComponentId').apply(lambda group: [{'code': row['mapTarget'],
                                                                                                    'mapGroup': row['mapPriority'],
                                                                                                    'mapPriority': row['mapPriority'],
                                                                                                    'mapBlock': row['mapBlock'],
                                                                                                    'mapAdvice': row['mapAdvice']} for _, row in group.iterrows()]).to_dict()

### Save for MedCAT

In [ ]:
# Save to CSV for medcat CDB creation
df.to_csv("preprocessed_snomed.csv", index=False)

--------

# Part 2: Create a MedCAT CDB using SNOMED CT release files


These steps are also in the [create_cdb.py](medcat/1_create_model/create_cdb/create_cdb.py)

In [ ]:
# Import required packages
from medcat.cdb import CDB
from medcat.config import Config
from medcat.cdb_maker import CDBMaker

#### Create concept database (cdb)

In [ ]:
# First initialise the default configuration
config = Config()
config.general['spacy_model'] = 'en_core_web_md'
maker = CDBMaker(config)

In [ ]:
# Create an array containing CSV files that will be used to build our CDB
csv_path = ['preprocessed_snomed.csv']

# Create your CDB
cdb = maker.prepare_csvs(csv_path, full_build=True)

### Inspect your cdb

In [ ]:
print(cdb.name2cuis['epilepsy'])

In [ ]:
print(cdb.cui2preferred_name['84757009'])

In [ ]:
print(cdb.cui2names['84757009'])

#### Enrich with extra information and mapping

Mapping was created in [Mappings to inbuilt external terminologies](https://colab.research.google.com/drive/1yesqjMQwQH20Kl9w7siRGVaSWU0uI84W#scrollTo=Mappings_to_inbuilt_external_terminologies).
Here we use [ICD-10](https://colab.research.google.com/drive/1yesqjMQwQH20Kl9w7siRGVaSWU0uI84W#scrollTo=ICD_10) as an example.

In [ ]:
cdb.addl_info['cui2icd10'] = sctid2icd10

### Save your new SNOMED cdb

__tip:__ good practise to include the snomed release edition file name

In [ ]:
cdb.save("SNOMED_cdb.dat")